In [ ]:
%load_ext autoreload
%autoreload 2

# This notebook illustrates how to annotate a visium H&E image  


In this notebook, we will annotate the visium H&E image extracted directly from the mapped 10X folder. 
We will load the image, and annotate it using various annotation tools. There are multiple options for image annotations: (1) fully-automatic - based on gene expression level, or (2) semi-automatic - based sparse manual labeling. Both (1) and (2) are used as a basis for prediction based on a pixel classifier. Finally,  we would finalise the image annotation based on manual annotation correction.
Annotations will be saved with the extension .pkl.

In [ ]:
# initialisation 
import os
import panel as pn
import socket
import numpy as np
import tissue_tag as tt
import tissue_tag.annotation

os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*"
# host = '5011' # set the port to the value in the address bar when operating in farm
host = '8888' # when working locally e.g. desktop

In [ ]:
# set path
# here you can either read a single image (grayscale or RGB) or generate a virtual H&E from 2 images in the next cell
Path = '../' #directory of tissuetag repo
path = Path +'data/tissue_tag_minimal_example_visium/'

# Step 1 - Create de-novo annotations from gene expression (or not)

Load a visium image and downscale it to a more manageable size. `res_in_ppm` is the desired pixels per micron in the output.

In [ ]:
tt_object = tt.read_visium(spaceranger_dir_path=path, use_resolution='hires', in_tissue=True, ppm_out=0.5, plot=True)

In [ ]:
# define annotations 
annodict = {
    'Medulla': 'green',
    'Cortex': 'cyan',
    'Edge': 'brown',
    'HS': 'magenta',
    'Vessels': 'blue',
    'PVS': 'red',
    'Other': 'orange'
}
tt_object.annotation_map = annodict

### Add semi automatic annotations based on gene expression 
In this option we wil use a simple random forest pixel classifier to call the cortex and medullar regions of the H&E image. The training labels will be derived based on the spots that show the highest expression of the marker gene.

In [ ]:
import scanpy
adata = scanpy.read_visium(path, count_file='raw_feature_bc_matrix.h5')

In [ ]:
gene_markers = {
    'Medulla' : [('AIRE',30)], # AIRE is highly expressed in the medulla (the marker for medullar TEC type 2 cells)
    'Cortex' : [('ARPP21',300)] # ARPP21 is highly expressed in the cortex
}

r = 35 # radius in microns for labels (27 is 1:1 label to visium spot - 55um diameter) but here i wanted to get some more context

# generate training data from gene expression
tissue_tag.annotation.gene_labels_from_adata(
    adata=adata,
    tissue_tag_annotation= tt_object,
    gene_markers = gene_markers,
    diameter = r*2,
    override_labels=True,
    normalize=False,
    intensity_threshold=210 # Values may need to be changed depending on image
) # generate gene-marker-labels

tissue_tag.annotation.plot_labels(tt_object, alpha=0.5)

# Part 2 - Iterative annotation section

At this stage, you can choose whether to use datashader - `use_datashader=True` for rendering the image (recommended for large images/high-resolution annotation). While the annotation process is slower with datashader, loading would be reasonable. If the image is too large, without datashader, the image might not load or take an extremely long time to load. 

Annotation is done by creating convex shapes in single strokes, the pixels inside the convex region would be filled in the `update_annotator` step. it's recommended to use a mouse with a wheel for easy scrolling in and out.
*to remove a label the use can click on the label to remove and press the backspace key. 

In [ ]:
# use annotator to label tissue regions according to categories indicated above
annotator = tissue_tag.annotation.annotator(tt_object, use_datashader=True)
pn.io.notebook.show_server(annotator, notebook_url=f'localhost:'+host)
#annotator.servable()

In [ ]:
# This step fills in the shapes created for the pixel classifier
tissue_tag.annotation.plot_labels(tt_object, alpha=0.5)

In [ ]:
%%time
# Train and predict the image pixels with a random forest classifier. This step takes about 1 to 10 min depending on number of training areas and resolution  
test_tt_object = tissue_tag.annotation.pixel_label_classifier(tt_object, classifier="LogisticRegression", threshold=0.9, downsampling_factor=4, plot=False, copy=True)
tissue_tag.annotation.plot_labels(test_tt_object, alpha=0.5)

From this point go back to the annotator and correct annotations untill happy with results.

# Part 3 - Finalise annotations and save

In [ ]:
tissue_tag.annotation.plot_labels(tt_object, alpha=0.5)

#  Save annotations (and load)

The resulting images and information can be saved for later use.

In [ ]:
isExist = os.path.exists(path+'tissue_annotations')
if not(isExist):
    os.mkdir(path+'/tissue_annotations/')
    
tt_object.save_annotation(file_path=path+'/tissue_annotations/annotations.h5')

In [ ]:
# optional - load annotations and as an intermediate step 
tt_object = tt.load_annotation(file_path=path + '/tissue_annotations/annotations.h5')